# MediaPipe 3D Face Transform Code 1

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_iris_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# MediaPipe 3D Face Transform Code 2

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Initialize MediaPipe face mesh and drawing utilities
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
drawing_spec = mp_drawing.DrawingSpec(thickness=1, color=(0, 255, 0))

# Set the transformation matrix (example: scaling)
transformation_matrix = np.array([[1.5, 0, 0],
                                  [0, 1.5, 0],
                                  [0, 0, 1]])

def transform_3d_face(image, landmarks):
    # Perform 3D face transformation
    transformed_landmarks = np.matmul(landmarks, transformation_matrix.T)
    transformed_image = image.copy()

    for i in range(transformed_landmarks.shape[0]):
        x, y, _ = transformed_landmarks[i]

        x = int(x * image.shape[1])
        y = int(y * image.shape[0])

        transformed_landmarks[i] = [x, y, _]

        cv2.circle(transformed_image, (x, y), 1, (255, 0, 0), -1)

    return transformed_image

# Initialize webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Flip the image horizontally for a mirror effect
    image = cv2.flip(image, 1)
    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image with MediaPipe face mesh
    results = face_mesh.process(image_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Convert face landmarks to a NumPy array
            landmarks = np.zeros((468, 3), dtype=np.float32)
            for i, landmark in enumerate(face_landmarks.landmark):
                landmarks[i] = [landmark.x, landmark.y, landmark.z]

            # Perform 3D face transformation
            transformed_image = transform_3d_face(image, landmarks)

            # Draw the face mesh on the image
            mp_drawing.draw_landmarks(
                transformed_image,
                face_landmarks,
                mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec)

    # Show the transformed image
    cv2.imshow('MediaPipe 3D Face Transform', transformed_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# MediaPipe 3D Face Transform Code 3

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Initialize MediaPipe face mesh and drawing utilities
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
drawing_spec = mp_drawing.DrawingSpec(thickness=1, color=(0, 255, 0))

def transform_3d_face(image, landmarks, replacement_face):
    # Perform 3D face transformation
    transformed_image = image.copy()

    # Get the bounding box coordinates of the face region
    xmin, ymin, xmax, ymax = get_face_bbox(landmarks, image.shape[:2])

    # Resize the replacement face image to match the face region
    resized_replacement_face = cv2.resize(replacement_face, (xmax - xmin, ymax - ymin))

    # Create a mask from the grayscale image of the replacement face
    mask = cv2.cvtColor(resized_replacement_face, cv2.COLOR_BGR2GRAY) / 255.0

    # Remove the alpha channel from the replacement face image
    replacement_face_rgb = resized_replacement_face[:, :, :3]

    # Calculate the region of interest for the replacement face
    roi = transformed_image[ymin:ymax, xmin:xmax]

    # Apply the replacement face to the region of interest
    roi = roi * (1 - mask[:, :, np.newaxis]) + replacement_face_rgb * mask[:, :, np.newaxis]

    # Place the modified region of interest back into the image
    transformed_image[ymin:ymax, xmin:xmax] = roi

    return transformed_image

def get_face_bbox(landmarks, image_shape):
    x_coordinates = [landmark[0] for landmark in landmarks]
    y_coordinates = [landmark[1] for landmark in landmarks]

    xmin = int(min(x_coordinates) * image_shape[1])
    ymin = int(min(y_coordinates) * image_shape[0])
    xmax = int(max(x_coordinates) * image_shape[1])
    ymax = int(max(y_coordinates) * image_shape[0])

    return xmin, ymin, xmax, ymax

# Initialize webcam
cap = cv2.VideoCapture(0)

# Load the replacement face image from webcam
ret, replacement_face = cap.read()

# Check if the replacement face image is captured successfully
if not ret:
    print("Failed to capture replacement face image from webcam.")
    exit()

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Flip the image horizontally for a mirror effect
    image = cv2.flip(image, 1)
    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image with MediaPipe face mesh
    results = face_mesh.process(image_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Convert face landmarks to a list of tuples
            landmarks = [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark]

            # Perform 3D face transformation
            transformed_image = transform_3d_face(image, landmarks, replacement_face)

            # Draw the face mesh on the image
            mp_drawing.draw_landmarks(
                transformed_image,
                face_landmarks,
                mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec)

    # Create a composite image with the sketch on the left and the original frame on the right
    composite_image = np.hstack((replacement_face, transformed_image))

    # Show the composite image
    cv2.imshow('MediaPipe 3D Face Transform', composite_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# MediaPipe 3D Face Transform Code 4

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Initialize MediaPipe pose and drawing utilities
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

# Set the range of colors to replace (in BGR format)
lower_color = np.array([0, 0, 0])  # lower range of color to replace
upper_color = np.array([50, 50, 50])  # upper range of color to replace

# Set the new color to replace the existing color (in BGR format)
new_color = np.array([0, 0, 255])  # new color (red)

# Initialize webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Flip the image horizontally for a mirror effect
    image = cv2.flip(image, 1)

    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image with MediaPipe pose detection
    results = pose.process(image_rgb)

    # Draw the pose landmarks on the image
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, drawing_spec, drawing_spec)

        # Get the coordinates of the left and right wrists
        left_wrist = (int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x * image.shape[1]),
                      int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y * image.shape[0]))
        right_wrist = (int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x * image.shape[1]),
                       int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y * image.shape[0]))

        # Create a mask for the range of colors to replace within the region of clothing
        mask = cv2.inRange(image, lower_color, upper_color)

        # Apply the new color to the mask
        colored_mask = cv2.bitwise_and(image, image, mask=mask)
        colored_mask[mask > 0] = new_color

        # Combine the original image and the colored mask
        result = cv2.bitwise_or(image, colored_mask)

        # Show the modified image
        cv2.imshow('Change Cloth Color', result)

    else:
        # Show the original image if no pose landmarks are detected
        cv2.imshow('Change Cloth Color', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()